In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [83]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [88]:
patient_df = pd.read_csv('data/allpatients_imputed_df').drop(columns=['Unnamed: 0','Unit1','Unit2'])

In [107]:
patient_df[patient_df.pid=="p00001"].head()
patient_df.shape
patient_df.HospAdmTime.unique().shape

,pid,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,HospAdmTime,ICULOS,SepsisLabel
179972,p00001,93.0,92.5,36.5,110.0,76.0,56.0,22.0,NaN,NaN,...,11.3,NaN,10.8,NaN,170.0,73,1,-214.64,1,0
179973,p00001,93.0,92.5,36.5,110.0,76.0,56.0,22.0,NaN,NaN,...,11.3,NaN,10.8,NaN,170.0,73,1,-214.64,2,0
179974,p00001,91.0,96.0,36.5,108.0,84.5,72.0,23.5,NaN,NaN,...,11.3,NaN,10.8,NaN,170.0,73,1,-214.64,3,0
179975,p00001,93.0,98.0,36.5,123.0,87.0,61.0,21.0,NaN,NaN,...,11.3,NaN,10.8,NaN,170.0,73,1,-214.64,4,0
179976,p00001,93.0,95.0,36.5,110.0,81.0,70.0,20.0,NaN,NaN,...,11.3,NaN,10.8,NaN,170.0,73,1,-214.64,5,0


(188453, 40)

(2976,)

In [120]:
patient_df = patient_df.fillna(patient_df.mean()['HR':'Platelets'])

In [121]:
patient_df[patient_df.pid=="p00001"].head()

,pid,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,HospAdmTime,ICULOS,SepsisLabel
179972,p00001,93.0,92.5,36.5,110.0,76.0,56.0,22.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,1,0
179973,p00001,93.0,92.5,36.5,110.0,76.0,56.0,22.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,2,0
179974,p00001,91.0,96.0,36.5,108.0,84.5,72.0,23.5,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,3,0
179975,p00001,93.0,98.0,36.5,123.0,87.0,61.0,21.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,4,0
179976,p00001,93.0,95.0,36.5,110.0,81.0,70.0,20.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,5,0


In [122]:
patient_df.pid.unique().shape

(5000,)

In [123]:
re_idx = patient_df.set_index(["pid","ICULOS"])
arr = re_idx.groupby("pid").apply(lambda x: x.values).values

train, test = train_test_split(arr, test_size=0.2)

In [126]:
train[0].shape
train[0][0]

(15, 38)

array([ 70.        , 100.        ,  36.3       , 146.        ,
       105.        ,  82.        ,  22.        ,  33.51751507,
        -3.16091766,  22.72753865,   0.5       ,   7.41      ,
        42.3       ,  99.        , 116.31360923,  13.        ,
        91.34764342,   1.13      , 105.84992634,   1.12      ,
         0.85336036, 115.        ,   4.02      ,   1.9       ,
         5.        ,   4.65      ,   1.44430028,   5.67129436,
        31.3       ,  10.7       ,  40.80889022,   7.8       ,
       296.93274106, 151.        ,  26.        ,   1.        ,
       -28.36      ,   0.        ])

In [127]:
# create windowing system here

In [87]:
df = pd.DataFrame(np.array([[12,1,1,1],[12,2,2,2],[12,3,2,2],[13,1,3,3],[13,2,4,4],[14,1,5,5],[14,2,6,6],[15,1,5,5]]), columns=["id","los","b","c"])
df.set_index(["id","los"], inplace=True)
df
arr = df.groupby("id").apply(lambda x: x.values).values
train, test = train_test_split(arr, test_size=0.2)
train
test

b  c
id los      
12 1    1  1
   2    2  2
   3    2  2
13 1    3  3
   2    4  4
14 1    5  5
   2    6  6
15 1    5  5

array([array([[5, 5],
       [6, 6]]),
       array([[1, 1],
       [2, 2],
       [2, 2]]),
       array([[3, 3],
       [4, 4]])], dtype=object)

array([array([[5, 5]])], dtype=object)